In [ ]:
import numpy as np
from astropy.io import fits
import matplotlib.pyplot as plt
import astropy.convolution

import lsst.daf.butler as dafButler

import lsst.afw.display as afwDisplay
afwDisplay.setDefaultBackend('firefly')